In [1]:
import polars as pl
import altair as alt
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [2]:
# load data
prepared_df = pl.read_parquet("../clean-datasets/transactions_prepared.parquet")

# display the dataframe
prepared_df

user_id,card_index,trx_amount,trx_method,merch_name,merch_city,merch_state,zip_code,merch_category_code,error_status,is_fraud,timestamp_transaction,weekday,hour,minute
i64,i64,f64,cat,str,str,str,str,cat,cat,str,datetime[μs],i8,i8,i8
0,0,134.09,"""Swipe Transact…","""35272132461278…","""La Verne""","""CA""","""9175""","""5300""","""No Error""","""No""",2002-09-01 06:21:00,7,6,21
0,0,38.48,"""Swipe Transact…","""-7276120921399…","""Monterey Park""","""CA""","""91754""","""5411""","""No Error""","""No""",2002-09-01 06:42:00,7,6,42
0,0,120.34,"""Swipe Transact…","""-7276120921399…","""Monterey Park""","""CA""","""91754""","""5411""","""No Error""","""No""",2002-09-02 06:22:00,1,6,22
0,0,128.95,"""Swipe Transact…","""34145274595791…","""Monterey Park""","""CA""","""91754""","""5651""","""No Error""","""No""",2002-09-02 17:45:00,1,17,45
0,0,104.71,"""Swipe Transact…","""58172184461787…","""La Verne""","""CA""","""9175""","""5912""","""No Error""","""No""",2002-09-03 06:23:00,2,6,23
0,0,86.19,"""Swipe Transact…","""-7146670748125…","""Monterey Park""","""CA""","""91755""","""5970""","""No Error""","""No""",2002-09-03 13:53:00,2,13,53
0,0,93.84,"""Swipe Transact…","""-7276120921399…","""Monterey Park""","""CA""","""91754""","""5411""","""No Error""","""No""",2002-09-04 05:51:00,3,5,51
0,0,123.5,"""Swipe Transact…","""-7276120921399…","""Monterey Park""","""CA""","""91754""","""5411""","""No Error""","""No""",2002-09-04 06:09:00,3,6,9
0,0,61.72,"""Swipe Transact…","""-7276120921399…","""Monterey Park""","""CA""","""91754""","""5411""","""No Error""","""No""",2002-09-05 06:14:00,4,6,14


In [3]:
fraud_distribution = (prepared_df
                      .group_by("is_fraud")
                      .agg([
                          pl.count("trx_amount").alias("count"),
                          pl.mean("trx_amount").alias("mean"),
                          pl.median("trx_amount").alias("median"),
                          pl.quantile("trx_amount", 0.25).alias("25th percentile"),
                          pl.quantile("trx_amount", 0.75).alias("75th percentile"),
                          pl.min("trx_amount").alias("min"),
                          pl.max("trx_amount").alias("max"),
                          pl.std("trx_amount").alias("std_dev"),
                          pl.col("trx_amount").skew().alias("skewness"),
                          pl.col("trx_amount").kurtosis().alias("kurtosis")
                      ]))

print("Fraud Transaction Amount Distribution")
fraud_distribution

Fraud Transaction Amount Distribution


is_fraud,count,mean,median,25th percentile,75th percentile,min,max,std_dev,skewness,kurtosis
str,u32,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""No""",24357143,53.958625,36.14,12.21,72.0,0.0,12390.5,75.277927,7.452958,160.615966
"""Yes""",29757,129.880318,79.51,25.44,165.0,0.0,5694.44,188.125601,6.984654,103.003114


In [4]:
prepared_df = prepared_df.with_columns([
    pl.col("timestamp_transaction").dt.strftime("%Y-%m-%d %H:%M:%S").alias("timestamp_transaction_str")
])

In [5]:
trx_method_fraud_distribution = (prepared_df
                                 .group_by(["trx_method", "is_fraud"])
                                 .agg(pl.count().alias("transaction_count"))
                                 .sort(["trx_method", "is_fraud"]))

print("Transaction Method vs Fraud Distribution")
trx_method_fraud_distribution

Transaction Method vs Fraud Distribution


trx_method,is_fraud,transaction_count
cat,str,u32
"""Swipe Transact…","""No""",15379510
"""Swipe Transact…","""Yes""",6572
"""Online Transac…","""No""",2694871
"""Online Transac…","""Yes""",18349
"""Chip Transacti…","""No""",6282762
"""Chip Transacti…","""Yes""",4836


In [13]:
# filter only fraud transactions
only_fraud_trx = prepared_df.filter(pl.col("is_fraud")=="Yes")

# take only two columns (timestamp_transaction_str and is_fraud) for efficiency
alt.Chart(only_fraud_trx.select("timestamp_transaction_str", "is_fraud")).mark_rect().encode(
    alt.X('hours(timestamp_transaction_str):O', title='Hour of Day'), # timeunit transformation
    alt.Y('day(timestamp_transaction_str):O', title='Day'), # timeunit transformation
    alt.Color('count(is_fraud):Q', title='Count', scale=alt.Scale(scheme='reds')), # aggregate function
    tooltip=[alt.Tooltip('hours(timestamp_transaction_str):O', title='Hour'),
             alt.Tooltip('day(timestamp_transaction_str):O', title='Day'),
             alt.Tooltip('count(is_fraud):Q', title='Count')]
).properties(
    title='Distribution of Fraudulent Transactions by Hour and Day',
    width=500,
    height=300
)

alt.Chart(...)

In [14]:
only_fraud_trx.head()

user_id,card_index,trx_amount,trx_method,merch_name,merch_city,merch_state,zip_code,merch_category_code,error_status,is_fraud,timestamp_transaction,weekday,hour,minute,timestamp_transaction_str
i64,i64,f64,cat,str,str,str,str,cat,cat,str,datetime[μs],i8,i8,i8,str
0,0,287.13,"""Online Transac…","""-8194607650924…","""ONLINE""","""NA""","""NA""","""3001""","""No Error""","""Yes""",2015-11-15 12:55:00,7,12,55,"""2015-11-15 12:…"
0,0,2.41,"""Online Transac…","""-7759074308363…","""ONLINE""","""NA""","""NA""","""5651""","""No Error""","""Yes""",2015-11-15 13:19:00,7,13,19,"""2015-11-15 13:…"
0,0,50.81,"""Online Transac…","""-5513321072133…","""ONLINE""","""NA""","""NA""","""4411""","""No Error""","""Yes""",2015-11-16 09:41:00,1,9,41,"""2015-11-16 09:…"
0,0,248.36,"""Online Transac…","""48723405188404…","""ONLINE""","""NA""","""NA""","""5732""","""No Error""","""Yes""",2015-11-16 09:46:00,1,9,46,"""2015-11-16 09:…"
0,0,473.0,"""Online Transac…","""-8566951830324…","""ONLINE""","""NA""","""NA""","""3640""","""No Error""","""Yes""",2015-11-16 11:20:00,1,11,20,"""2015-11-16 11:…"


In [27]:
# aggregate overall transactions by time
total_transactions_by_time = (prepared_df
                              .with_columns(pl.col("timestamp_transaction").dt.hour().alias("hour"))
                              .with_columns(pl.col("timestamp_transaction").dt.weekday().alias("weekday"))
                              .group_by(["hour", "weekday"])
                              .agg(pl.count().alias("total_transactions"))
                              .sort(["weekday", "hour"]))

# aggregate fraud transactions by time
fraud_transactions_by_time = (prepared_df.filter(pl.col("is_fraud") == "Yes")
                              .with_columns(pl.col("timestamp_transaction").dt.hour().alias("hour"))
                              .with_columns(pl.col("timestamp_transaction").dt.weekday().alias("weekday"))
                              .group_by(["hour", "weekday"])
                              .agg(pl.count().alias("fraud_transactions"))
                              .sort(["weekday", "hour"]))

# merge and calc the ratio
transaction_ratios = (total_transactions_by_time
                      .join(fraud_transactions_by_time, on=["hour", "weekday"], how="left")
                      .with_columns((pl.col("fraud_transactions") / pl.col("total_transactions")).alias("fraud_ratio")))

transaction_ratios.head()

hour,weekday,total_transactions,fraud_transactions,fraud_ratio
i8,i8,u32,u32,f64
0,1,33440,26,0.000778
1,1,30419,27,0.000888
2,1,30715,41,0.001335
3,1,25964,60,0.002311
4,1,29207,85,0.00291


In [26]:
alt.Chart(transaction_ratios).mark_rect().encode(
    alt.X('hour:O', title='Hour of Day'),
    alt.Y('weekday:O', title='Day of Week'),
    alt.Color('fraud_ratio:Q', title='Fraud Ratio', scale=alt.Scale(scheme='reds')),
    tooltip=[
        alt.Tooltip('hour:O', title='Hour'),
        alt.Tooltip('weekday:O', title='Day of Week'),
        alt.Tooltip('fraud_ratio:Q', title='Fraud Ratio')
    ]
).properties(
    title='Fraud Ratio by Hour and Day of Week',
    width=500,
    height=300
)

alt.Chart(...)